In [63]:
import findspark
findspark.init()
import pyspark

In [64]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [65]:
data = spark.read.csv("G:/titanic.csv ",inferSchema=True,header=True)

In [66]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [67]:
data.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [68]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [69]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [70]:
final_data = my_cols.na.drop()
final_data.count()

712

## Convert the Categorical data into Vector using OneHotEncoder and StringIndexer :

### => Convert Sex and Embarked columns using StringIndexer() and OneHotEncoder() respectively

In [71]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer

In [72]:
sexIndexer = StringIndexer(inputCol="Sex", outputCol="Sex_index")
sexEncoder = OneHotEncoder(inputCol="Sex_index",outputCol="sexVector")

In [73]:
EmbIndexer = StringIndexer(inputCol="Embarked", outputCol="Embarked_index")
EmbEncoder = OneHotEncoder(inputCol="Embarked_index",outputCol="EmbarkedVector")

In [74]:
assembler = VectorAssembler(inputCols=['Pclass','sexVector','Age','SibSp','Parch','EmbarkedVector'],outputCol='feature')

## Pipeline

In [75]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [76]:
LogReg = LogisticRegression(featuresCol='feature',labelCol='Survived')

In [77]:
pipeline = Pipeline(stages=[sexIndexer,sexEncoder,EmbIndexer,EmbEncoder,assembler,LogReg])

In [78]:
train, test = final_data.randomSplit([0.7,0.3])

In [79]:
fitting_model = pipeline.fit(train)

In [80]:
result = fitting_model.transform(test)

## Binary Classification Evaluator

In [81]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [82]:
my_eval = BinaryClassificationEvaluator(labelCol='Survived',rawPredictionCol='prediction')

In [83]:
result.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
+--------+----------+
only showing top 20 rows



In [84]:
AUC = my_eval.evaluate(result)

In [85]:
AUC

0.7444673668417103

## Result :

### The logistic regression model seems to be performing decently with area under the curve of 0.75